In [4]:
import numpy as np
import pandas as pd

In [9]:
class Recommend():
    
    def __init__(self,similarity,user,N):
        self.similiarity = similarity
        self.user = user
        self.N = N
        self.factor = 0.9
        
    def pearsonCorrelationSimilarity(A, B):
        if len(A) < 3: return 1
        else: return 0.5 + 0.5 * np.corrcoef(A, B, rowvar=0)[0][1]
    
    def cosineSimilarity(A, B):
        num = float(A.T*B)
        denom = np.linalg.norm(A) * np.linalg.norm(B)
        return 0.5 + 0.5 * (num/denom)
    
    def loadData(self):
        ratings_matrix = pd.read_csv('./datasets/MS_ratings_matrix.csv', index_col=0) # the first column is index
        return ratings_matrix,np.mat(ratings_matrix)
    # loadData()
    
    def getSingularValues(Sigma,factor=0.9):
        square  = 0
        energy = 0
        for si in Sigma:
            square = si*si
            energy = energy+square 
        threshold =energy*factor
        energy = 0
        for i, si in enumerate(Sigma):
            sqaure = si * si
            energy += sqaure
            if energy > threshold:
                break
        return i

In [15]:
r = Recommend("cosine",100,5)
data,matrix = r.loadData()
print(data)


       X1000  X1001  X1002  X1003  X1004  X1005  X1006  X1007  X1008  X1009  \
10001      1      1      1      0      0      0      0      0      0      0   
10002      0      1      0      1      0      0      0      0      0      0   
10003      0      1      0      1      1      0      0      0      0      0   
10004      0      0      0      0      0      1      0      0      0      0   
10005      0      0      0      0      0      0      1      0      0      0   
...      ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
42707      0      0      0      0      1      0      0      0      1      1   
42708      0      0      0      0      0      0      0      0      1      0   
42709      0      1      0      1      0      0      0      0      0      0   
42710      0      1      0      0      0      0      0      0      0      0   
42711      0      0      0      0      0      0      0      0      1      0   

       ...  X1275  X1276  X1277  X1278  X1279  X128

In [16]:
U, Sigma, VT = np.linalg.svd(matrix, full_matrices=False)
# i = r.getSingularValues(Sigma)